In [ ]:
import sys

sys.path[1:1] = ["_common", "_common/qiskit"]
sys.path[1:1] = ["../../_common", "../../_common/qiskit"]
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
import time
import math
import numpy as np
np.random.seed(0)
import execute as ex
import metrics as metrics
from collections import defaultdict

In [ ]:
from qiskit.quantum_info import SparsePauliOp
from qiskit_algorithms import TimeEvolutionProblem, SciPyRealEvolver

In [ ]:
# Benchmark Name
benchmark_name = "Hamiltonian Simulation"

np.random.seed(0)

verbose = False

# Saved circuits and subcircuits for display
QC_ = None
XX_ = None
YY_ = None
ZZ_ = None
XXYYZZ_ = None

# For validating the implementation of XXYYZZ operation
_use_XX_YY_ZZ_gates = False


def initial_state(n_spins: int, method: int) -> QuantumCircuit:
    """
    Initialize the quantum state.
    
    Args:
        n_spins (int): Number of spins (qubits).
        method (int): Method of initialization (1 for checkerboard state, otherwise GHZ state).

    Returns:
        QuantumCircuit: The initialized quantum circuit.
    """
    qc = QuantumCircuit(n_spins)

    if method == 1:
        # Checkerboard state, or "Neele" state
        for k in range(0, n_spins, 2):
            qc.x([k])
    else:
        # GHZ state: 1/sqrt(2) (|00...> + |11...>)
        qc.h(0)
        for k in range(1, n_spins):
            qc.cx(k-1, k)

    return qc

def construct_TFIM_hamiltonian(n_spins: int) -> SparsePauliOp:
    """
    Construct the Transverse Field Ising Model (TFIM) Hamiltonian.

    Args:
        n_spins (int): Number of spins (qubits).

    Returns:
        SparsePauliOp: The Hamiltonian represented as a sparse Pauli operator.
    """
    pauli_strings = []
    coefficients = []
    g = 0.2  # Strength of the transverse field

    # Pauli spin vector product terms
    for i in range(n_spins):
        x_term = 'I' * i + 'X' + 'I' * (n_spins - i - 1)
        pauli_strings.append(x_term)
        coefficients.append(g)

    identity_string = ['I'] * n_spins

    # ZZ operation on each pair of qubits in a linear chain
    for j in range(2):
        for i in range(j % 2, n_spins - 1, 2):
            zz_term = identity_string.copy()
            zz_term[i] = 'Z'
            zz_term[(i + 1) % n_spins] = 'Z'
            zz_term = ''.join(zz_term)
            pauli_strings.append(zz_term)
            coefficients.append(1.0)

    return SparsePauliOp.from_list(zip(pauli_strings, coefficients))

def construct_heisenberg_hamiltonian(n_spins: int, w : float, h_x: list[float], h_z: list[float]) -> SparsePauliOp:
    """
    Construct the Heisenberg Hamiltonian with disorder.

    Args:
        n_spins (int): Number of spins (qubits).

    Returns:
        SparsePauliOp: The Hamiltonian represented as a sparse Pauli operator.
    """

    pauli_strings = []
    coefficients = []

    # Disorder terms
    for i in range(n_spins):
        x_term = 'I' * i + 'X' + 'I' * (n_spins - i - 1)
        z_term = 'I' * i + 'Z' + 'I' * (n_spins - i - 1)
        pauli_strings.append(x_term)
        coefficients.append(w * h_x[i])
        pauli_strings.append(z_term)
        coefficients.append(w * h_z[i])

    identity_string = ['I'] * n_spins

    # Interaction terms
    for j in range(2):
        for i in range(j % 2, n_spins - 1, 2):
            xx_term = identity_string.copy()
            yy_term = identity_string.copy()
            zz_term = identity_string.copy()

            xx_term[i] = 'X'
            xx_term[(i + 1) % n_spins] = 'X'

            yy_term[i] = 'Y'
            yy_term[(i + 1) % n_spins] = 'Y'

            zz_term[i] = 'Z'
            zz_term[(i + 1) % n_spins] = 'Z'

            pauli_strings.append(''.join(xx_term))
            coefficients.append(1.0)
            pauli_strings.append(''.join(yy_term))
            coefficients.append(1.0)
            pauli_strings.append(''.join(zz_term))
            coefficients.append(1.0)

    return SparsePauliOp.from_list(zip(pauli_strings, coefficients))

def construct_hamiltonian(n_spins: int, method: int, w : float, h_x: list[float], h_z: list[float]) -> SparsePauliOp:
    """
    Construct the Hamiltonian based on the specified method.

    Args:
        n_spins (int): Number of spins (qubits).
        method (int): Method of Hamiltonian construction (1 for Heisenberg, 2 for TFIM).

    Returns:
        SparsePauliOp: The constructed Hamiltonian.
    """
    if method == 1:
        return construct_heisenberg_hamiltonian(n_spins, w, h_x, h_z)
    elif method == 2:
        return construct_TFIM_hamiltonian(n_spins)
    else:
        raise ValueError("Method is not equal to 1 or 2.")

def HamiltonianSimulationExact(n_spins: int, t: float, method: int, w : float, h_x: list[float], h_z: list[float]) -> dict:
    """
    Perform exact Hamiltonian simulation using classical matrix evolution.

    Args:
        n_spins (int): Number of spins (qubits).
        t (float): Duration of simulation.
        method (int): Method of Hamiltonian construction (1 for Heisenberg, 2 for TFIM).

    Returns:
        dict: The distribution of the evolved state.
    """
    hamiltonian = construct_hamiltonian(n_spins, method, w, h_x, h_z)
    time_problem = TimeEvolutionProblem(hamiltonian, t, initial_state=initial_state(n_spins, method))
    result = SciPyRealEvolver(num_timesteps=1).evolve(time_problem)
    return result.evolved_state.probabilities_dict()

def HamiltonianSimulation(n_spins: int, K: int, t: float, method: int, w : float, h_x: list[float], h_z: list[float]) -> QuantumCircuit:
    """
    Construct a Qiskit circuit for Hamiltonian simulation.

    Args:
        n_spins (int): Number of spins (qubits).
        K (int): The Trotterization order.
        t (float): Duration of simulation.
        method (int): Method of Hamiltonian construction (1 for Heisenberg, 2 for TFIM).

    Returns:
        QuantumCircuit: The constructed Qiskit circuit.
    """
    num_qubits = n_spins
    secret_int = f"{K}-{t}"
    
    # Allocate qubits
    qr = QuantumRegister(n_spins)
    cr = ClassicalRegister(n_spins)
    qc = QuantumCircuit(qr, cr, name=f"hamsim-{num_qubits}-{secret_int}")
    tau = t / K

    if method == 1:
        # Checkerboard state, or "Neele" state
        for k in range(0, n_spins, 2):
            qc.x([k])

        # Loop over each Trotter step, adding gates to the circuit defining the Hamiltonian
        for k in range(K):
            # Pauli spin vector product
            [qc.rx(2 * tau * w * h_x[i], qr[i]) for i in range(n_spins)]
            [qc.rz(2 * tau * w * h_z[i], qr[i]) for i in range(n_spins)]
            qc.barrier()
            
            # Basic implementation of exp(i * t * (XX + YY + ZZ))
            if _use_XX_YY_ZZ_gates:
                for j in range(2):
                    for i in range(j % 2, n_spins - 1, 2):
                        qc.append(xx_gate(tau).to_instruction(), [qr[i], qr[(i + 1) % n_spins]])
                        qc.append(yy_gate(tau).to_instruction(), [qr[i], qr[(i + 1) % n_spins]])
                        qc.append(zz_gate(tau).to_instruction(), [qr[i], qr[(i + 1) % n_spins]])
            else:
                # Optimized XX + YY + ZZ operator on each pair of qubits in linear chain
                for j in range(2):
                    for i in range(j % 2, n_spins - 1, 2):
                        qc.append(xxyyzz_opt_gate(tau).to_instruction(), [qr[i], qr[(i + 1) % n_spins]])
            qc.barrier()
    elif method == 2:
        g = 0.2  # Strength of transverse field

        # GHZ state: 1/sqrt(2) (|00...> + |11...>)
        qc.h(qr[0])
        for k in range(1, n_spins):
            qc.cx(qr[k-1], qr[k])
        qc.barrier()

        # Calculate TFIM
        for k in range(K):
            for i in range(n_spins):
                qc.rx(2 * tau * g, qr[i])
            qc.barrier()

            for j in range(2):
                for i in range(j % 2, n_spins - 1, 2):
                    qc.append(zz_gate(tau).to_instruction(), [qr[i], qr[(i + 1) % n_spins]])
            qc.barrier()

        # Reverse transformation from GHZ state
        for k in reversed(range(1, n_spins)):
            qc.cx(qr[k-1], qr[k])
        qc.h(qr[0])
        qc.barrier()
    else:
        raise ValueError("Invalid method specification.")

    # Measure all qubits
    for i_qubit in range(n_spins):
        qc.measure(qr[i_qubit], cr[i_qubit])

    # Save smaller circuit example for display
    global QC_
    if QC_ is None or n_spins <= 6:
        if n_spins < 9:
            QC_ = qc

    return qc

############### XX, YY, ZZ Gate Implementations

def xx_gate(tau: float) -> QuantumCircuit:
    """
    Simple XX gate on q0 and q1 with angle 'tau'.

    Args:
        tau (float): The rotation angle.

    Returns:
        QuantumCircuit: The XX gate circuit.
    """
    qr = QuantumRegister(2)
    qc = QuantumCircuit(qr, name="xx_gate")
    qc.h(qr[0])
    qc.h(qr[1])
    qc.cx(qr[0], qr[1])
    qc.rz(3.1416 * tau, qr[1])
    qc.cx(qr[0], qr[1])
    qc.h(qr[0])
    qc.h(qr[1])
    
    global XX_
    XX_ = qc
    
    return qc

def yy_gate(tau: float) -> QuantumCircuit:
    """
    Simple YY gate on q0 and q1 with angle 'tau'.

    Args:
        tau (float): The rotation angle.

    Returns:
        QuantumCircuit: The YY gate circuit.
    """
    qr = QuantumRegister(2)
    qc = QuantumCircuit(qr, name="yy_gate")
    qc.s(qr[0])
    qc.s(qr[1])
    qc.h(qr[0])
    qc.h(qr[1])
    qc.cx(qr[0], qr[1])
    qc.rz(3.1416 * tau, qr[1])
    qc.cx(qr[0], qr[1])
    qc.h(qr[0])
    qc.h(qr[1])
    qc.sdg(qr[0])
    qc.sdg(qr[1])

    global YY_
    YY_ = qc

    return qc

def zz_gate(tau: float) -> QuantumCircuit:
    """
    Simple ZZ gate on q0 and q1 with angle 'tau'.

    Args:
        tau (float): The rotation angle.

    Returns:
        QuantumCircuit: The ZZ gate circuit.
    """
    qr = QuantumRegister(2)
    qc = QuantumCircuit(qr, name="zz_gate")
    qc.cx(qr[0], qr[1])
    qc.rz(3.1416 * tau, qr[1])
    qc.cx(qr[0], qr[1])

    global ZZ_
    ZZ_ = qc

    return qc

def xxyyzz_opt_gate(tau: float) -> QuantumCircuit:
    """
    Optimal combined XXYYZZ gate (with double coupling) on q0 and q1 with angle 'tau'.

    Args:
        tau (float): The rotation angle.

    Returns:
        QuantumCircuit: The optimal combined XXYYZZ gate circuit.
    """
    alpha = tau
    beta = tau
    gamma = tau
    qr = QuantumRegister(2)
    qc = QuantumCircuit(qr, name="xxyyzz_opt")
    qc.rz(3.1416 / 2, qr[1])
    qc.cx(qr[1], qr[0])
    qc.rz(3.1416 * gamma - 3.1416 / 2, qr[0])
    qc.ry(3.1416 / 2 - 3.1416 * alpha, qr[1])
    qc.cx(qr[0], qr[1])
    qc.ry(3.1416 * beta - 3.1416 / 2, qr[1])
    qc.cx(qr[1], qr[0])
    qc.rz(-3.1416 / 2, qr[0])

    global XXYYZZ_
    XXYYZZ_ = qc

    return qc

In [ ]:
import json
from qiskit_aer import Aer
from qiskit import transpile

w = 1 
k = 5
t = .2 # Heisenburg state evolves "just" enough 

min_qubits = 2
max_qubits = 11

backend = Aer.get_backend("qasm_simulator")

precalculated_data = {}

# store parameters in precalculated data
precalculated_data["w"] = w
precalculated_data["k"] = k
precalculated_data["t"] = t

# add parameter random values to precalculated data to ensure consistency
np.random.seed(26)
precalculated_data['h_x'] = list(2 * np.random.random(20) - 1) # random numbers between [-1, 1]
np.random.seed(75)
precalculated_data['h_z'] = list(2 * np.random.random(20) - 1) # random numbers between [-1, 1]

num_shots = 100000

for n_spins in range(min_qubits, max_qubits+1):

        print(f"Now running n_spins {n_spins}")

        h_x = precalculated_data['h_x'][:n_spins]
        h_z = precalculated_data['h_z'][:n_spins]

        qc = HamiltonianSimulation(n_spins, k, t, method=1, w=w, h_x = hx, h_z = hz)

        qc3 = HamiltonianSimulation(n_spins, k, t, method=2, w=w, h_x = hx, h_z = hz)

        transpiled_qc = transpile(qc, backend, optimization_level=0)
        job = backend.run(transpiled_qc, shots=num_shots)
        result = job.result()
        counts = result.get_counts(qc)

        transpiled_qc3 = transpile(qc3, backend, optimization_level=0)
        job3 = backend.run(transpiled_qc3, shots=num_shots)
        result3 = job3.result()
        counts3 = result3.get_counts()

        dist = {}
        for key in counts.keys():
            prob = counts[key] / num_shots
            dist[key] = prob

        dist3 = {}
        for key in counts3.keys():
            prob = counts3[key] / num_shots
            dist3[key] = prob

        # add dist values to precalculated data for use in fidelity calculation
        precalculated_data[f"Heisenburg - Qubits{n_spins}"] = dist  
        precalculated_data[f"Exact Heisenburg - Qubits{n_spins}"] = HamiltonianSimulationExact(n_spins, t=t, method=1, w=w, h_x = hx, h_z = hz)
        precalculated_data[f"TFIM - Qubits{n_spins}"] = dist3 
        precalculated_data[f"Exact TFIM - Qubits{n_spins}"] = HamiltonianSimulationExact(n_spins, t=t, method=2, w=w, h_x = hx, h_z = hz) 

# https://stackoverflow.com/questions/36021332/how-to-prettyprint-human-readably-print-a-python-dict-in-json-format-double-q
with open('precalculated_data.json', 'w') as f:
    f.write(json.dumps(
        precalculated_data,
        sort_keys=True,
        indent=4,
        separators=(',', ': ')
        ))